# devlog 2023-08-11

_author: Tyler Coles_

To wrap up movement optimization, lets do some simple benchmarking to compare the performance of the two engines (BasicEngine and HypercubeEngine) under different conditions.

In [20]:
import concurrent.futures
import time
from datetime import date
from typing import NamedTuple

from epymorph.data import geo_library, ipm_library, mm_library
from epymorph.movement.basic import BasicEngine
from epymorph.movement.hypercube import HypercubeEngine
from epymorph.simulation import Simulation

uber_params = {
    "alpha": [0.1, 0.3, 0.2],
    "beta": 0.4,
    "gamma": 0.25,  # 1/4
    "xi": 0.111,  # 1/90
    "phi": 40.0,
    "omega": [0.55, 0.05],
    "delta": [0.333, 0.5, 0.166, 0.142, 0.125],
    "gamma_sparsemod": [0.166, 0.333, 0.25],
    "rho": [0.40, 0.175, 0.015, 0.20, 0.60],
    "theta": 0.1,
    "move_control": 0.9,
    "infection_duration": 4.0,
    "immunity_duration": 90.0,
    "hospitalization_duration": 14.0,
    "hospitalization_rate": 0.1,
    "infection_seed_loc": 0,
    "infection_seed_size": 10000,
}


class Args(NamedTuple):
    ipm: str
    mm: str
    geo: str
    days: int


# Here are the scenarios we will run: each with Basic and Hypercube.
jobs = [
    # Just a basic SIRS model on a single population with no movement.
    Args("sirs", "no", "single_pop", 150),
    # A null IPM but a moderate-sized geo with movement.
    Args("no", "centroids", "us_states_2015", 150),
    # The most basic real experiment: Pei.
    Args("pei", "pei", "pei", 150),
    # A complex IPM with a moderate geo.
    Args("sparsemod", "centroids", "us_states_2015", 150),
    # A very large geo with a real IPM.
    Args("sirs", "centroids", "maricopa_cbg_2019", 30),
]


class Result(NamedTuple):
    id: int
    args: Args
    basic_time: float
    hypercube_time: float


def simulate(id: int, args: Args) -> Result:
    ipm, mm, geo, days = args
    ps = uber_params.copy()
    if ipm == "sparsemod":
        ps["gamma"] = ps["gamma_sparsemod"]

    sim1 = Simulation(
        geo=geo_library[geo](),
        ipm_builder=ipm_library[ipm](),
        mvm_builder=mm_library[mm](),
        mvm_engine=BasicEngine,
    )
    t0 = time.perf_counter()
    sim1.run(ps, date(2019, 1, 1), days)
    t1 = time.perf_counter()
    del sim1  # make sure we're not keeping memory tied down

    sim2 = Simulation(
        geo=geo_library[geo](),
        ipm_builder=ipm_library[ipm](),
        mvm_builder=mm_library[mm](),
        mvm_engine=HypercubeEngine,
    )
    t2 = time.perf_counter()
    sim2.run(ps, date(2019, 1, 1), days)
    t3 = time.perf_counter()
    del sim2  # make sure we're not keeping memory tied down

    return Result(id, args, t1 - t0, t3 - t2)


results = []
with concurrent.futures.ProcessPoolExecutor(max_workers=2) as exec:
    futures = {exec.submit(simulate, id, args): id for id, args in enumerate(jobs)}
    done, _ = concurrent.futures.wait(futures)
    for f in done:
        id = futures[f]
        try:
            results.append(f.result())
        except Exception as e:
            print(f"Worker {id} raised an exception: {repr(e)}")

display(results)

[Result(id=3, args=Args(ipm='pei', mm='pei', geo='pei', days=150), basic_time=0.3412574620006126, hypercube_time=0.4623291640000389),
 Result(id=0, args=Args(ipm='sirs', mm='no', geo='single_pop', days=150), basic_time=0.03445045700027549, hypercube_time=0.0490778160001355),
 Result(id=2, args=Args(ipm='sparsemod', mm='centroids', geo='us_states_2015', days=150), basic_time=5.59075156200015, hypercube_time=5.595188520999727),
 Result(id=4, args=Args(ipm='sirs', mm='centroids', geo='maricopa_cbg_2019', days=30), basic_time=89.11756116599918, hypercube_time=45.72292028499942),
 Result(id=1, args=Args(ipm='no', mm='centroids', geo='us_states_2015', days=150), basic_time=0.6852275270002792, hypercube_time=0.579345139999532)]

(This took me about 2 minutes to run.) Now we can jam the results in a dataframe and calculate the relative speedup/slowdown factor.

$$speedup = \frac{time_{baseline}}{time_{experimental}}$$

In [23]:
import pandas as pd

df = pd.DataFrame.from_records(
    results, index="id", columns=["id", "args", "basic", "hypercube"]
).sort_index()

df["speedup"] = df["basic"] / df["hypercube"]

df

,args,basic,hypercube,speedup
id,,,,
0,"(sirs, no, single_pop, 150)",0.034450,0.049078,0.701956
1,"(no, centroids, us_states_2015, 150)",0.685228,0.579345,1.182762
2,"(sparsemod, centroids, us_states_2015, 150)",5.590752,5.595189,0.999207
3,"(pei, pei, pei, 150)",0.341257,0.462329,0.738127
4,"(sirs, centroids, maricopa_cbg_2019, 30)",89.117561,45.722920,1.949079


Clearly HypercubeEngine is not always an improvement! It does seem to perform better for very large GEOs, but only by a factor of 2. To explain the lack of overall improvement, I believe Hypercube does save time processing movement but requires more time processing the IPM. Interfacing Hypercube's world representation with the IPM layer requires copying to intermediary arrays and this takes time. The IPM is currently naive to the underlying data representation and thus cannot optimize its processing accordingly. Future work could address this by making the "engine" concept holistic rather than limited to the movement system.

Overall I would call this effort a success. It involved not only introducing the concept of MovementEngine, rewriting the old object-based way of handling movement (as BasicEngine), and adding the HypercubeEngine -- it also involved improvements upon BasicEngine's algorithm. The original baseline `(sirs, centroids, maricopa_cbg_2019, 30)` took around 319 seconds; so we've still achieved a speedup of 3.5 and 7.0 (for Basic and Hypercube respectively) for this case.